Goal: Within five minutes of getting the raw file, have a quality analysis.
Most of the data the Kelly lab looks at is from the Max Quant output.
This generally takes hours to produce, which is ineffective for QC.

I am experimenting with doing it in pyteomics or pymzml.
These cannot open raw, but can use .mzML.


###### Step 1: use MSConvertGUI to convert .raw to .mzML
Or use MSConvert.exe via command line.
Time: 2 minutes

In [1]:
test_file='D:\\MS\\QC\\QC_Run3.mzML'#This is the one we're checking.
standard_file='D:\MS\QC\QC_Run1.mzML'#This should be a known good run.

In [2]:
import pymzml
import numpy
from statistics import mean
from datetime import datetime

In [3]:
print(datetime.now())
run=pymzml.run.Reader(test_file)
comp_run=pymzml.run.Reader(standard_file)
r_length = run.get_spectrum_count()
comp_length = comp_run.get_spectrum_count()
mid = int(mean([r_length, comp_length])/2)
start_index = mid - 250
end_index = mid + 250
print(datetime.now())
sim=[]
for i in range(start_index,end_index, 5):
    spec = run[i]
    comp_spec = comp_run[i]
    similarity_score = spec.similarity_to(comp_spec)
    sim.append(similarity_score)
print ("Similarity:\t{0:.2f}%".format((mean(sim))*100))
print(datetime.now())

2019-08-08 16:28:03.922145
D:\MS\QC\QC_Run3.mzML
D:\MS\QC\QC_Run1.mzML
2019-08-08 16:28:03.933404
Similarity:	16.00%
2019-08-08 16:28:15.636355


In [20]:
from pyteomics import mzml, auxiliary
import numpy as np
import pandas as pd
test_reader = mzml.read(test_file)
standard_reader = mzml.read(standard_file)
auxiliary.print_tree(next(test_reader))

index
id
defaultArrayLength
scanList
 -> count
 -> scan [list]
 ->  -> scanWindowList
 ->  ->  -> count
 ->  ->  -> scanWindow [list]
 ->  ->  ->  -> scan window lower limit
 ->  ->  ->  -> scan window upper limit
 ->  -> scan start time
 ->  -> filter string
 ->  -> ion injection time
 -> no combination
MS1 spectrum
ms level
positive scan
profile spectrum
base peak m/z
base peak intensity
total ion current
lowest observed m/z
highest observed m/z
spectrum title
count
m/z array
intensity array


In [21]:
spec = test_reader.next()

In [35]:
spec

{'index': 1,
 'id': 'controllerType=0 controllerNumber=1 scan=2',
 'defaultArrayLength': 37781,
 'scanList': {'count': 1,
  'scan': [{'scanWindowList': {'count': 1,
     'scanWindow': [{'scan window lower limit': 375.0 m/z,
       'scan window upper limit': 1575.0 m/z}]},
    'scan start time': 0.0110184016 minute,
    'filter string': 'FTMS + p NSI Full ms [375.0000-1575.0000]',
    'ion injection time': 14.744358062744 millisecond}],
  'no combination': ''},
 'MS1 spectrum': '',
 'ms level': 1,
 'positive scan': '',
 'profile spectrum': '',
 'base peak m/z': 471.796081542969 m/z,
 'base peak intensity': 4404172.0 number of detector counts,
 'total ion current': 64723296.0,
 'lowest observed m/z': 371.268572486683 m/z,
 'highest observed m/z': 1590.868620533731 m/z,
 'spectrum title': 'Nanowell_TMT10_SingleCell_9channel_boost0_run2_031218_305bar.2.2. File:"Nanowell_TMT10_SingleCell_9channel_boost0_run2_031218_305bar.raw", NativeID:"controllerType=0 controllerNumber=1 scan=2"',
 'count

In [ ]:
stats_to_check=[
    'base peak m/z',
    'base peak intensity'
]

def compare_stats(t, ref, stats_to_check):
    for s in stats_to_check:
        r = t[s]/ref[s]
        stats[s]=r
        print ("{0}:\t{1:.2f}% match.".format(s, stats[s]*100))